In [1]:
#@title 1. Install Dependencies (~1 min)
%%capture
![ ! -d SPfast ] && git clone https://github.com/tlitfin/SPfast.git
%pip install pybind11
%pip install itables
!cd SPfast/src && make gnu
%pip install SPfast/
%pip install biopython
![ ! -f mkdssp-4.4.0-linux-x64 ] && wget https://github.com/PDB-REDO/dssp/releases/download/v4.4.0/mkdssp-4.4.0-linux-x64 && chmod +x mkdssp-4.4.0-linux-x64

In [2]:
#@title 2. Download AFDB-clusters SPfast structure files (~2 min)
%%capture
![ -f afdb-clu-annot.db ] || wget https://spfast.tomlitfin.workers.dev/afdb-clu-annot.db.tar.gz
!tar xvzf afdb-clu-annot.db.tar.gz && rm afdb-clu-annot.db.tar.gz
!wget https://spfast.tomlitfin.workers.dev/SPfast-protein2ipr-afdbclust-PFAM-complex.tsv.gz && gunzip SPfast-protein2ipr-afdbclust-PFAM-complex.tsv.gz

#PFAM clans
![ -f Pfam-A.clans.tsv ] || wget https://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.clans.tsv.gz
![ -f Pfam-A.clans.tsv ] || gunzip Pfam-A.clans.tsv.gz

In [3]:
#@title 3. Select parameters for SPfast search
#%%capture # Can't capture with file-upload
from google.colab import files
from pathlib import Path

#@markdown #Query
UniProt_ID = "A0A0U5EPG3" # @param {type:"string"}
#@markdown **OR**
PDB_ID = "" # @param {type:"string"}
Chain = "" # @param {type:"string"}
#@markdown **OR**

#@markdown Leave ID fields blank for custom **monomer** upload in PDB format
#@markdown  - The *first* chain will be extracted from a multi-chain query

#@markdown #Search Parameters
Optimization_objective = 'SPfscore' # @param ["SPfscore", "SPscore"]
#score_cutoff = 0.4 # @param {type:"number"}
#d0 = 4.0 # @param {type:"number"}
#finalgap0 = 0.2 # @param {type:"number"}
#alpha = 0.3 # @param {type:"number"}
#coarsecut = -1.0 # @param {type:"number"}
#segcut = 5.0 # @param {type:"number"}
fast = True # @param {type:"boolean"}
trim = True # @param {type:"boolean"}
#@markdown #Annotation Parameters
Top_ranked_hits = 200 # @param {type:"integer"}
#@markdown - Number of top-ranking templates to evaluate
PFAM_coverage = 0.8 # @param {type:"slider", min:0.6, max:1, step:0.01}
#@markdown - Minimum coverage of PFAM annotated domain for function transfer
#Similarity_cutoff = 0.6 # @param {type:"slider", min:0.4, max:1, step:0.01}
##@markdown - Minimum structure similarity score for PFAM annotated domain template

if UniProt_ID == "" and PDB_ID == "":
  uploaded = files.upload()
  fn = list(uploaded.keys())[0]
  !python SPfast/utils/extract_chain.py {fn}
  #HACKY WORKAROUND for DSSP - may not be required always
  !cat <(echo "HEADER    SPFAST-SEARCH                           01-JAN-25   1ABC") {fn} > tmpfile && mv tmpfile {fn}

  AFDB_ID = Path(fn).stem
elif UniProt_ID != "":
  AFDB_ID = f'AF-{UniProt_ID}-F1-model_v4'
  ![ ! -f {AFDB_ID}.pdb ] && wget https://alphafold.ebi.ac.uk/files/{AFDB_ID}.pdb &> /dev/null
  !grep -v "^DBREF" {AFDB_ID}.pdb > tmp.pdb && mv tmp.pdb {AFDB_ID}.pdb
elif PDB_ID != "" and Chain != "":
  AFDB_ID = f'{PDB_ID}_{Chain}'
  ![ ! -f {PDB_ID}.pdb ] && wget https://files.rcsb.org/download/{PDB_ID}.pdb &> /dev/null
  !python SPfast/utils/extract_chain.py {PDB_ID}.pdb {Chain}
  #HACKY WORKAROUND for DSSP - may not be required always
  !cat <(echo "HEADER    SPFAST-SEARCH                           01-JAN-25   {PDB_ID}") {AFDB_ID}.pdb > tmpfile && mv tmpfile {AFDB_ID}.pdb

![ ! -d {AFDB_ID} ] && mkdir {AFDB_ID}
fast_flag = ''
sp_flag = ''
if fast:
  fast_flag = '-fast'

if Optimization_objective == 'SPscore':
  sp_flag = '-SPscore'


In [4]:
#@title 4. Prepare query structure
#%%capture
!./mkdssp-4.4.0-linux-x64 "{AFDB_ID}.pdb" --output-format=dssp > {AFDB_ID}/{AFDB_ID}.dssp
if trim:
  !python SPfast/utils/idealize.py <(echo {AFDB_ID}) --dssdir {AFDB_ID}/ --sdir ./ --odir {AFDB_ID}/ --af2model --trim
else:
  !python SPfast/utils/idealize.py <(echo {AFDB_ID}) --dssdir {AFDB_ID}/ --sdir ./ --odir {AFDB_ID}/ --af2model
!./SPfast/src/prepare_bin.gnu -q {AFDB_ID}/{AFDB_ID}.ideal

In [5]:
#@title 5. Search annotated AFDB-clusters database
#@markdown Table shows **top hits** during search - *full results available for download*
import pandas as pd
from math import ceil
from itables import show
from IPython.display import clear_output, HTML, display

def display_dat(fn):
  with open(fn) as f:
    df = pd.read_csv(f, delimiter=' ', names=['query', 'db', 'score', 'raw', 'ss_prefilter', 'q_len', 'db_len', 'eff_len', 'seqid', 'ali_len', 'seeds', 'pass_seeds', 'seg_score'])
  show(df[['query', 'db', 'score', 'seqid', 'q_len', 'db_len', 'ali_len']], order=[[2]], maxBytes=0, lengthMenu=[10, 20, 100])

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

![ -f {AFDB_ID}/{AFDB_ID}.sp1 ] && rm {AFDB_ID}/{AFDB_ID}.sp1
![ -f {AFDB_ID}/display.sp1 ] && rm {AFDB_ID}/display.sp1
![ -f {AFDB_ID}/tmp1.sp1 ] && rm {AFDB_ID}/tmp1.sp1
out = display(progress(0, 100), display_id=True)

total_size = 394431
shard_size = ceil(total_size/100)

![ -f {AFDB_ID}/{AFDB_ID}.txt ] && rm {AFDB_ID}/{AFDB_ID}.txt

i0=0
!touch {AFDB_ID}/display.sp1
while i0<total_size:
  batchstart=i0
  batchend=min(i0+shard_size, total_size)
  #print(batchstart, batchend)

  # Run search
  !SPfast/src/SPfast.gnu -q {AFDB_ID}/{AFDB_ID}.ideal.bin -tdb afdb-clu-annot.db -batchstart {batchstart} -batchend {batchend} {sp_flag} {fast_flag} -ssprefcut -1. -iprint 3 | sed 's/\.ideal\.bin//g' | sed 's/\.ideal//g' > {AFDB_ID}/tmp1.sp3

  # Top hits to display
  !cat {AFDB_ID}/tmp1.sp3 >> {AFDB_ID}/{AFDB_ID}.txt
  !cat <(awk 'NR%7==1' {AFDB_ID}/tmp1.sp3 ) {AFDB_ID}/display.sp1 | sort -rnk3 | head -n {Top_ranked_hits} > {AFDB_ID}/tmp2.sp1
  !mv {AFDB_ID}/tmp2.sp1 {AFDB_ID}/display.sp1

  # Update display
  clear_output()
  out.update(progress(min(100, 100*(batchend)/total_size), 100))
  print(f"{batchend}/{total_size} structures searched ({round(100*batchend/total_size,2)}%)")
  display_dat(f'{AFDB_ID}/display.sp1')
  i0+=shard_size
  #if i0>50000: break
  #break #for testing

# Return entire output
#!sort -rnk3 {AFDB_ID}.sp1 -o {AFDB_ID}.txt



394431/394431 structures searched (100.0%)


In [6]:
#@title 6. Map structure to top PFAM clans
import SPlib
from collections import defaultdict, Counter
import operator

#Map InterPro to PFAM
interpro_map = {}
#Uniprot ID to InterPro (start and end)
protein2ipr_data = defaultdict(list)
with open('SPfast-protein2ipr-afdbclust-PFAM-complex.tsv') as f:
    f.readline()
    for line in f:
        ss = line.split('\t')
        protein2ipr_data[ss[0]].append((ss[1], int(ss[4]), int(ss[5])))
        interpro_map[ss[1]] = ss[3]

#Map family to clan
clan_map = {}
with open('Pfam-A.clans.tsv') as f:
    for line in f:
        ss = line.split('\t')
        if ss[2] == '':
            clan_map[ss[0]] = ss[3]
        else:
            clan_map[ss[0]] = ss[2]

db_hits = set()
with open(f'{AFDB_ID}/display.sp1') as f:
  for line in f:
    ss = line.split()
    name0 = ss[1]
    db_hits.add(name0)

spfn = f'{AFDB_ID}/{AFDB_ID}.ideal.bin'

query = SPlib.Protein2(spfn)
secstart = query.getsegstart()
secend = query.getsegend()
ssec = query.getssec()
segdict = {}
for i, (sstart, send) in enumerate(zip(secstart, secend)):
    for zz in range(sstart, send+1):
        segdict[zz] = i
#print(segdict)
counter = Counter()
data0 = []

toskip = True
with open(f'{AFDB_ID}/{AFDB_ID}.txt') as f:
  for i, line in enumerate(f):
    ss = line.split()
    line_idx = i%7
    if line_idx>0 and toskip:
      continue
    elif line_idx==0:
        if ss[1] not in db_hits:
          toskip = True
          continue
        else:
          toskip = False
        afdbid = ss[1]
        name = ss[1].split('-')[1]
        score = float(ss[2])
    elif line_idx==4:
        start1 = int(ss[0])
        seq1 = ss[1] #gapped
    elif line_idx==5:
        matches = line[5:].strip('\n')
    elif line_idx==6:
        if len(protein2ipr_data.get(name,[]))==0:continue #shouldn't be possible
        seq2 = ss[1] #gapped
        indices = []
        j=0
        i=0
        seglist = []
        segtypeset = set()
        start = int(ss[0])

        for char1, char2, aln_match in zip(seq1, seq2, matches):
            if char2 == '-': i+=1; continue
            if char1 == '-': j+=1; continue
            if aln_match in ':;.': #alignment up to 8A
                indices.append(start+j)
                if (start1+i) in segdict:
                    seglist.append(segdict[start1+i])
                    segtypeset.add(ssec[start1+i])
            j+=1
            i+=1

        aligned_segcount = Counter(seglist)
        aligned_segs = sum([1 for v in aligned_segcount.values() if v>=3])
        #print(name, score, aligned_segs)
        if aligned_segs<5 or (aligned_segs<6 and 2 not in segtypeset): continue #high complexity

        for fam_dat in protein2ipr_data.get(name, []):
            fam, start, end = fam_dat
            n_hits = len([idx for idx in indices if start <= idx <= end])
            #print(afdbid, fam, score, clan_map.get(interpro_map.get(fam,fam), interpro_map.get(fam,fam)), n_hits/(end+1-start), start, end, n_hits)
            if n_hits/(end+1-start)>PFAM_coverage: #coverage
                #print(afdbid, fam, n_hits/(end+1-start), start, end)
                data0.append((name, score, fam, n_hits/(end+1-start)))


target_fams = set()
results = []
clan_counter = defaultdict(int)
for d in sorted(data0, key=operator.itemgetter(1), reverse=True):
    clan = clan_map.get(interpro_map.get(d[2],d[2]),interpro_map.get(d[2],d[2]))
    clan_counter[clan]+=1
    if clan in target_fams: continue
    results.append((AFDB_ID, d[0], "%.3f"%d[1], "%.3f"%d[3], d[2], interpro_map.get(d[2],d[2]), clan))
    target_fams.add(clan)

#for q, t, s, cov, i, c, clan in results:
#  print(q, t, s, cov, i, c, clan, clan_counter[clan])

headers = ["Query", "Template", "Score", "PFAM coverage", "InterPro", "PFAM", "Clan"]
df = pd.DataFrame(results, columns=headers)
df["Count"] = df['Clan'].apply(lambda x: clan_counter[x])
df.to_csv(f"{AFDB_ID}/{AFDB_ID}-PFAM.csv")
show(df, order=[[2]], maxBytes=0)

In [7]:
#@title 7. Download result files
#%%capture
![ -f {AFDB_ID}/{AFDB_ID}_SPfast.tar.gz ] && rm {AFDB_ID}/{AFDB_ID}_SPfast.tar.gz
!tar cvzf {AFDB_ID}/{AFDB_ID}_SPfast.tar.gz {AFDB_ID}/{AFDB_ID}.txt {AFDB_ID}/{AFDB_ID}-PFAM.csv
files.download(f"{AFDB_ID}/{AFDB_ID}_SPfast.tar.gz")
#files.download(f"{AFDB_ID}.txt.gz")
#files.download(f"{AFDB_ID}-PFAM.csv")

AF-A0A0U5EPG3-F1-model_v4/AF-A0A0U5EPG3-F1-model_v4.txt
AF-A0A0U5EPG3-F1-model_v4/AF-A0A0U5EPG3-F1-model_v4-PFAM.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>